In [1]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import random

# connexion avec la base de données
from pymongo import MongoClient

In [2]:
#URL = 'https://fr.indeed.com/jobs?q=data%20scientist&l=Paris%20(75)&start=20&vjk=3d5487bc57a59ba4'


In [5]:
url_template = "https://fr.indeed.com/jobs?q={}&l={}&start={}&sort=date"
max_results_per_city =10 # Set this to a high-value (5000) to generate more results. 

sleep_time = random.randrange(0,10)

In [6]:
i = 0
results = []
df_more = pd.DataFrame(columns=["Title","Location","Company","Rating","Salary", "Synopsis"])

for metier in set(['developpeur', 'data+scientist', 'data+analyst', 'business+intelligence']):    
    for start in range(0, 1000, 10):

        # Grab the results from the request (as above)
        url = url_template.format(metier, 'Paris', start)
        # Append to the full set of results
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        for each in soup.find_all(class_= "result" ):
            time.sleep(sleep_time)

            try:
                title = each.find('span', title=True).text.replace('\n', '')
            except:
                title = 'None'
            try:
                location = each.find('div', {'class':"companyLocation" }).text.replace('\n', '')
            except:
                location = 'None'
            try:
                company = each.find(class_='companyName').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('div', {'class':'salary-snippet'}).text
            except:
                salary = 'None'
            try:
                rating_span = each.find('span', attrs={'class':  'ratingNumber'})
                rating = float(rating_span.text.strip().replace(',', '.'))
            except:
                rating = 'None'
            try:
                if each.find('a',href=True):     

                    jk=each['data-jk']
                    #print(jk)
                    #https://fr.indeed.com/voir-emploi?jk=006167bb602a6f9b
                    url='https://fr.indeed.com/voir-emploi?jk='+(jk)
                    #print(url)


                    job_response = requests.get(url)
                    job_soup = BeautifulSoup(job_response.content, 'html.parser')
                    synopsis = job_soup.find('div', {'class' : 'jobsearch-jobDescriptionText'}).text
            except:
                synopsis = None

            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Rating':rating, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
            i += 1
            if i % 100 == 0:  # Ram helped me build this counter to see how many. You can visibly see Ram's vernacular in the print statements.
                print('You have ' + str(i) + ' results. ' + str(df_more.dropna().drop_duplicates().shape[0]) + " of these aren't rubbish.")    


You have 100 results. 92 of these aren't rubbish.
You have 200 results. 183 of these aren't rubbish.
You have 300 results. 254 of these aren't rubbish.
You have 400 results. 304 of these aren't rubbish.
You have 500 results. 383 of these aren't rubbish.
You have 600 results. 441 of these aren't rubbish.
You have 700 results. 526 of these aren't rubbish.
You have 800 results. 595 of these aren't rubbish.
You have 900 results. 694 of these aren't rubbish.
You have 1000 results. 780 of these aren't rubbish.
You have 1100 results. 862 of these aren't rubbish.
You have 1200 results. 910 of these aren't rubbish.
You have 1300 results. 979 of these aren't rubbish.
You have 1400 results. 1052 of these aren't rubbish.
You have 1500 results. 1114 of these aren't rubbish.
You have 1600 results. 1203 of these aren't rubbish.
You have 1700 results. 1286 of these aren't rubbish.
You have 1800 results. 1369 of these aren't rubbish.
You have 1900 results. 1438 of these aren't rubbish.
You have 2000 re

In [7]:
df_more.shape

(2541, 6)

In [8]:
df_more.to_csv('Indeed_Project_3_df_more_long_not_cleaned_paris.csv', encoding='utf-8')